In [1]:
import numpy as np

ge = np.array([])
laplace_det = np.array([])
nop = np.array([])
sum_p_norm = np.array([])
sum_s_norm = np.array([])
sum_f_norm = np.array([])
sampling_det = np.array([])
path_norm = np.array([])
sigma = np.array([])

#,'tanh'
for act in ['relu','tanh']:
    for net in ['alexnet_1','alexnet_2','alexnet_3']:
        fname = net+'_'+act+'/'
        ge = np.append(ge,np.load(fname+'ge'+'.npy',allow_pickle=True))
        laplace_det = np.append(laplace_det,np.load(fname+'laplace_det'+'.npy',allow_pickle=True))
        nop = np.append(nop,np.load(fname+'nop'+'.npy',allow_pickle=True))
        sum_p_norm = np.append(sum_p_norm,np.load(fname+'sum_p_norm'+'.npy',allow_pickle=True))
        sum_s_norm = np.append(sum_s_norm,np.load(fname+'sum_s_norm'+'.npy',allow_pickle=True))
        sum_f_norm = np.append(sum_f_norm,np.load(fname+'sum_f_norm'+'.npy',allow_pickle=True))
        sampling_det = np.append(sampling_det,np.load(fname+'sampling_det'+'.npy',allow_pickle=True))
        sigma = np.append(sigma,np.load(fname+'sigma'+'.npy',allow_pickle=True))          
        path_norm = np.append(path_norm,np.load(fname+'path_norm'+'.npy',allow_pickle=True))  

In [2]:
def kendall(a,b):
    b = np.array(b)
    pos = 0
    neg = 0
    for i in range(len(a)-1):
        for j in range(i+1,len(a)):
            if (a[i]-a[j])*(b[i]-b[j])>0:
                pos += 1
            elif (a[i]-a[j])*(b[i]-b[j])<0:
                neg += 1
    if pos+neg == 0:
        return 0
    else:
        return (pos-neg)/(pos+neg)

def mi(a,b):
    b = np.array(b)
    index = np.arange(len(a))
    np.random.shuffle(index)
    a = a[index]
    b = b[index]
    h1 = 0
    h2 = 0
    h3 = 0
    h4 = 0
    for i in range(len(a)-1):
        for j in range(i+1,len(a)):
            if (a[i]-a[j])>0 and (b[i]-b[j])>0.0001:
                h1 += 1
            elif (a[i]-a[j])>0 and (b[i]-b[j])<-0.0001:
                h2 += 1
            elif (a[i]-a[j])<0 and (b[i]-b[j])>0.0001:
                h3 += 1
            elif (a[i]-a[j])<0 and (b[i]-b[j])<-0.0001:
                h4 += 1
    h_all = h1+h2+h3+h4
    if h_all==0:
        return 0
    else:
        p1 = h1/h_all
        p2 = h2/h_all
        p3 = h3/h_all
        p4 = h4/h_all
        mi = 0
        if p1>0.:
            mi += p1*np.log2(p1/((p1+p2)*(p1+p3))) 
        if p2>0.:
            mi += p2*np.log2(p2/((p1+p2)*(p2+p4))) 
        if p3>0.:
            mi += p3*np.log2(p3/((p3+p4)*(p1+p3))) 
        if p4>0.:
            mi += p4*np.log2(p4/((p3+p4)*(p2+p4)))
        return mi

In [3]:

def subset2(ij):
    lr_ = [0.1, 0.03]
    drop_rate_ = [0.0, 0.15, 0.3]
    width_ = [1., 0.5]
    batch_size_ = [64, 256]
    weight_decay_ = [0.0, 0.0005]

    all_co = []
    all_mi = []

    all_co_act = []
    all_mi_act = []
    all_co_net = []
    all_mi_net = []
    all_co_weight_decay = []
    all_mi_weight_decay = []
    all_co_drop_rate = []
    all_mi_drop_rate = []
    all_co_width = []
    all_mi_width = []
    all_co_batch_size = []
    all_mi_batch_size = []
    all_co_lr = []
    all_mi_lr = []
    for ii in range(5):
        for jj in range(ii+1,6):
            i = 0
            dic_ge = dict()
            dic_laplace_det = dict()
            dic_nop = dict()
            dic_sum_p_norm = dict()
            dic_sum_s_norm = dict()
            dic_sum_f_norm = dict()
            dic_sampling_det = dict()
            dic_sigma = dict()
            dic_path_norm = dict()

            dic_act = dict()
            dic_net = dict()
            dic_weight_decay = dict()
            dic_drop_rate = dict()
            dic_width = dict()
            dic_batch_size = dict()
            dic_lr = dict()
            for act in ['relu','tanh']:
                for net in ['alexnet_1','alexnet_2','alexnet_3']:
                    for weight_decay in [0.0, 0.0005]:
                        for drop_rate in [0.0, 0.15, 0.3]:
                            for width in [1., 0.5]:
                                for batch_size in [64, 256]:
                                    for lr in [0.1, 0.03]:
                                        sub = [act, net, weight_decay, width, batch_size, lr]
                                        sub_a = str(sub[ii])
                                        sub_b = str(sub[jj])

                                        if sub_a+sub_b in dic_ge.keys():
                                            dic_laplace_det[sub_a+sub_b].append(laplace_det[i])
                                            dic_nop[sub_a+sub_b].append(nop[i])
                                            dic_ge[sub_a+sub_b].append(ge[i])
                                            dic_sum_p_norm[sub_a+sub_b].append(sum_p_norm[i])
                                            dic_sum_s_norm[sub_a+sub_b].append(sum_s_norm[i])
                                            dic_sum_f_norm[sub_a+sub_b].append(sum_f_norm[i])
                                            dic_sampling_det[sub_a+sub_b].append(sampling_det[i])
                                            dic_sigma[sub_a+sub_b].append(sigma[i])
                                            dic_path_norm[sub_a+sub_b].append(path_norm[i])

                                            dic_act[sub_a+sub_b].append(ord(act[0]))
                                            dic_net[sub_a+sub_b].append(ord(net[-1]))
                                            dic_weight_decay[sub_a+sub_b].append(weight_decay)
                                            dic_drop_rate[sub_a+sub_b].append(drop_rate)
                                            dic_width[sub_a+sub_b].append(width)
                                            dic_batch_size[sub_a+sub_b].append(batch_size)
                                            dic_lr[sub_a+sub_b].append(lr)
                                        else:
                                            dic_laplace_det[sub_a+sub_b] = [laplace_det[i]]
                                            dic_nop[sub_a+sub_b] = [nop[i]]
                                            dic_ge[sub_a+sub_b] = [ge[i]]
                                            dic_sum_p_norm[sub_a+sub_b] = [sum_p_norm[i]]
                                            dic_sum_s_norm[sub_a+sub_b] = [sum_s_norm[i]]
                                            dic_sum_f_norm[sub_a+sub_b] = [sum_f_norm[i]]
                                            dic_sampling_det[sub_a+sub_b] = [sampling_det[i]]
                                            dic_sigma[sub_a+sub_b] = [sigma[i]]
                                            dic_path_norm[sub_a+sub_b] = [path_norm[i]]

                                            dic_act[sub_a+sub_b] = [ord(act[0])]
                                            dic_net[sub_a+sub_b] = [ord(net[-1])]
                                            dic_weight_decay[sub_a+sub_b] = [weight_decay]
                                            dic_drop_rate[sub_a+sub_b] = [drop_rate]
                                            dic_width[sub_a+sub_b] = [width]
                                            dic_batch_size[sub_a+sub_b] = [batch_size]
                                            dic_lr[sub_a+sub_b] = [lr]
                                        i += 1   
            sub_name = ['act', 'net', 'weight_decay', 'width', 'batch_size', 'lr']
            co = []
            m_i = []

            co_act = []
            m_i_act = []
            co_net = []
            m_i_net = []
            co_weight_decay = []
            m_i_weight_decay = []
            co_drop_rate = []
            m_i_drop_rate = []
            co_width = []
            m_i_width = []
            co_batch_size = []
            m_i_batch_size = []
            co_lr = []
            m_i_lr = []

            for key in dic_ge:
                #test_m = np.array(dic_sum_f_norm[key])/(np.array(dic_sigma[key])*150000) + np.log2(1/np.array(dic_sampling_det[key]))

                #test_m = np.array(np.log2(1/np.array(dic_sampling_det[key])))
                if ij == 1:
                    test_m = np.array(dic_sum_f_norm[key])   
                elif ij == 2:
                    test_m = np.array(dic_sum_s_norm[key])
                elif ij == 3:
                    test_m = np.array(dic_sum_p_norm[key])
                elif ij == 4:
                    test_m = np.array(dic_path_norm[key])
                elif ij == 5:
                    test_m = np.array(dic_sigma[key])*2*np.log2(2*np.array(dic_nop[key]))
                elif ij == 6:
                    test_m = np.array(dic_sum_f_norm[key])/(np.array(dic_sigma[key])*150000)
                elif ij == 7:
                    test_m = np.array(dic_sum_f_norm[key])/(np.array(dic_sigma[key])*150000) + np.log2(1/np.array(dic_laplace_det[key]))
                elif ij == 8:
                    test_m = np.array(dic_sum_f_norm[key])/(np.array(dic_sigma[key])*150000) + np.log2(1/np.array(dic_sampling_det[key]))

                co.append(kendall(test_m,dic_ge[key]))
                m_i.append(mi(test_m,dic_ge[key]))

                '''if 'act' not in [sub_name[ii], sub_name[jj]]:
                    co_act.append(kendall(test_m,dic_act[key]))
                    m_i_act.append(mi(test_m,dic_act[key]))
                if 'net' not in [sub_name[ii], sub_name[jj]]:
                    co_net.append(kendall(test_m,dic_net[key]))
                    m_i_net.append(mi(test_m,dic_net[key]))
                if 'weight_decay' not in [sub_name[ii], sub_name[jj]]:
                    co_weight_decay.append(kendall(test_m,dic_weight_decay[key]))
                    m_i_weight_decay.append(mi(test_m,dic_weight_decay[key]))
                if 'drop_rate' not in [sub_name[ii], sub_name[jj]]:
                    co_drop_rate.append(kendall(test_m,dic_drop_rate[key]))
                    m_i_drop_rate.append(mi(test_m,dic_drop_rate[key]))
                if 'width' not in [sub_name[ii], sub_name[jj]]:
                    co_width.append(kendall(test_m,dic_width[key]))
                    m_i_width.append(mi(test_m,dic_width[key]))
                if 'batch_size' not in [sub_name[ii], sub_name[jj]]:
                    co_batch_size.append(kendall(test_m,dic_batch_size[key]))
                    m_i_batch_size.append(mi(test_m,dic_batch_size[key]))
                if 'lr' not in [sub_name[ii], sub_name[jj]]:
                    co_lr.append(kendall(test_m,dic_lr[key]))
                    m_i_lr.append(mi(test_m,dic_lr[key]))'''

            #print(sub_name[ii]+'_'+sub_name[jj])
            #print('corr ',np.mean(co)) 
            #print('mi ',np.mean(m_i)) 
            all_co.append(np.mean(co))
            all_mi.append(np.mean(m_i))

            '''if len(co_act)>0:
                all_co_act.append(np.mean(co_act))
                all_mi_act.append(np.mean(m_i_act))
            if len(co_net)>0:
                all_co_net.append(np.mean(co_net))
                all_mi_net.append(np.mean(m_i_net))
            if len(co_weight_decay)>0:
                all_co_weight_decay.append(np.mean(co_weight_decay))
                all_mi_weight_decay.append(np.mean(m_i_weight_decay))
            if len(co_drop_rate)>0:
                all_co_drop_rate.append(np.mean(co_drop_rate))
                all_mi_drop_rate.append(np.mean(m_i_drop_rate))
            if len(co_width)>0:
                all_co_width.append(np.mean(co_width))
                all_mi_width.append(np.mean(m_i_width))
            if len(co_batch_size)>0:
                all_co_batch_size.append(np.mean(co_batch_size))
                all_mi_batch_size.append(np.mean(m_i_batch_size))
            if len(co_lr)>0:
                all_co_lr.append(np.mean(co_lr))
                all_mi_lr.append(np.mean(m_i_lr))'''

    pr_co = []
    pr_mi = []

    '''cco = np.mean(all_co_batch_size) 
    mmi = np.mean(all_mi_batch_size) 
    pr_co.append('%.4f' % cco)
    pr_mi.append('%.4f' % mmi)

    cco = np.mean(all_co_lr)
    mmi = np.mean(all_mi_lr) 
    pr_co.append('%.4f' % cco)
    pr_mi.append('%.4f' % mmi)

    cco = np.mean(all_co_net)
    mmi = np.mean(all_mi_net) 
    pr_co.append('%.4f' % cco)
    pr_mi.append('%.4f' % mmi)

    cco = np.mean(all_co_act)
    mmi = np.mean(all_mi_act)
    pr_co.append('%.4f' % cco)
    pr_mi.append('%.4f' % mmi)

    cco = np.mean(all_co_weight_decay)
    mmi = np.mean(all_mi_weight_decay) 
    pr_co.append('%.4f' % cco)
    pr_mi.append('%.4f' % mmi)

    cco = np.mean(all_co_width) 
    mmi = np.mean(all_mi_width) 
    pr_co.append('%.4f' % cco)
    pr_mi.append('%.4f' % mmi)

    cco = np.mean(all_co_drop_rate)
    mmi = np.mean(all_mi_drop_rate) 
    pr_co.append('%.4f' % cco)
    pr_mi.append('%.4f' % mmi)'''

    cco = np.mean(all_co)
    mmi = np.mean(all_mi) 
    pr_co.append(cco)
    pr_mi.append(mmi)

    return all_mi, pr_mi


In [4]:
#########w==1###################
def subset1(ij):
    lr_ = [0.1, 0.03]
    drop_rate_ = [0.0, 0.15, 0.3]
    width_ = [1., 0.5]
    batch_size_ = [64, 256]
    weight_decay_ = [0.0, 0.0005]

    all_co = []
    all_mi = []

    all_co_act = []
    all_mi_act = []
    all_co_net = []
    all_mi_net = []
    all_co_weight_decay = []
    all_mi_weight_decay = []
    all_co_drop_rate = []
    all_mi_drop_rate = []
    all_co_width = []
    all_mi_width = []
    all_co_batch_size = []
    all_mi_batch_size = []
    all_co_lr = []
    all_mi_lr = []
    for ii in range(6):
        i = 0
        dic_ge = dict()
        dic_laplace_det = dict()
        dic_nop = dict()
        dic_sum_p_norm = dict()
        dic_sum_s_norm = dict()
        dic_sum_f_norm = dict()
        dic_sampling_det = dict()
        dic_sigma = dict()
        dic_path_norm = dict()

        dic_act = dict()
        dic_net = dict()
        dic_weight_decay = dict()
        dic_drop_rate = dict()
        dic_width = dict()
        dic_batch_size = dict()
        dic_lr = dict()
        for act in ['relu','tanh']:
            for net in ['vgg11','vgg16','vgg19']:
                for weight_decay in [0.0, 0.0005]:
                    for drop_rate in [0.0, 0.15, 0.3]:
                        for width in [1., 0.5]:
                            for batch_size in [64, 256]:
                                for lr in [0.1, 0.03]:
                                    sub = [act, net, weight_decay, width, batch_size, lr]
                                    sub_a = str(sub[ii])
                                    jj = ii
                                    sub_b = str(sub[jj])

                                    if sub_a+sub_b in dic_ge.keys():
                                        dic_laplace_det[sub_a+sub_b].append(laplace_det[i])
                                        dic_nop[sub_a+sub_b].append(nop[i])
                                        dic_ge[sub_a+sub_b].append(ge[i])
                                        dic_sum_p_norm[sub_a+sub_b].append(sum_p_norm[i])
                                        dic_sum_s_norm[sub_a+sub_b].append(sum_s_norm[i])
                                        dic_sum_f_norm[sub_a+sub_b].append(sum_f_norm[i])
                                        dic_sampling_det[sub_a+sub_b].append(sampling_det[i])
                                        dic_sigma[sub_a+sub_b].append(sigma[i])
                                        dic_path_norm[sub_a+sub_b].append(path_norm[i])

                                        dic_act[sub_a+sub_b].append(ord(act[0]))
                                        dic_net[sub_a+sub_b].append(ord(net[-1]))
                                        dic_weight_decay[sub_a+sub_b].append(weight_decay)
                                        dic_drop_rate[sub_a+sub_b].append(drop_rate)
                                        dic_width[sub_a+sub_b].append(width)
                                        dic_batch_size[sub_a+sub_b].append(batch_size)
                                        dic_lr[sub_a+sub_b].append(lr)
                                    else:
                                        dic_laplace_det[sub_a+sub_b] = [laplace_det[i]]
                                        dic_nop[sub_a+sub_b] = [nop[i]]
                                        dic_ge[sub_a+sub_b] = [ge[i]]
                                        dic_sum_p_norm[sub_a+sub_b] = [sum_p_norm[i]]
                                        dic_sum_s_norm[sub_a+sub_b] = [sum_s_norm[i]]
                                        dic_sum_f_norm[sub_a+sub_b] = [sum_f_norm[i]]
                                        dic_sampling_det[sub_a+sub_b] = [sampling_det[i]]
                                        dic_sigma[sub_a+sub_b] = [sigma[i]]
                                        dic_path_norm[sub_a+sub_b] = [path_norm[i]]

                                        dic_act[sub_a+sub_b] = [ord(act[0])]
                                        dic_net[sub_a+sub_b] = [ord(net[-1])]
                                        dic_weight_decay[sub_a+sub_b] = [weight_decay]
                                        dic_drop_rate[sub_a+sub_b] = [drop_rate]
                                        dic_width[sub_a+sub_b] = [width]
                                        dic_batch_size[sub_a+sub_b] = [batch_size]
                                        dic_lr[sub_a+sub_b] = [lr]
                                    i += 1   
        sub_name = ['act', 'net', 'weight_decay', 'width', 'batch_size', 'lr']
        co = []
        m_i = []

        co_act = []
        m_i_act = []
        co_net = []
        m_i_net = []
        co_weight_decay = []
        m_i_weight_decay = []
        co_drop_rate = []
        m_i_drop_rate = []
        co_width = []
        m_i_width = []
        co_batch_size = []
        m_i_batch_size = []
        co_lr = []
        m_i_lr = []

        for key in dic_ge:
            if ij == 1:
                test_m = np.array(dic_sum_f_norm[key])   
            elif ij == 2:
                test_m = np.array(dic_sum_s_norm[key])
            elif ij == 3:
                test_m = np.array(dic_sum_p_norm[key])
            elif ij == 4:
                test_m = np.array(dic_path_norm[key])
            elif ij == 5:
                test_m = np.array(dic_sigma[key])*2*np.log2(2*np.array(dic_nop[key]))
            elif ij == 6:
                test_m = np.array(dic_sum_f_norm[key])/(np.array(dic_sigma[key])*150000)
            elif ij == 7:
                test_m = np.array(dic_sum_f_norm[key])/(np.array(dic_sigma[key])*150000) + np.log2(1/np.array(dic_laplace_det[key]))
            elif ij == 8:
                test_m = np.array(dic_sum_f_norm[key])/(np.array(dic_sigma[key])*150000) + np.log2(1/np.array(dic_sampling_det[key]))

            co.append(kendall(test_m,dic_ge[key]))
            m_i.append(mi(test_m,dic_ge[key]))

            if 'act' not in [sub_name[ii], sub_name[jj]]:
                co_act.append(kendall(test_m,dic_act[key]))
                m_i_act.append(mi(test_m,dic_act[key]))
            if 'net' not in [sub_name[ii], sub_name[jj]]:
                co_net.append(kendall(test_m,dic_net[key]))
                m_i_net.append(mi(test_m,dic_net[key]))
            if 'weight_decay' not in [sub_name[ii], sub_name[jj]]:
                co_weight_decay.append(kendall(test_m,dic_weight_decay[key]))
                m_i_weight_decay.append(mi(test_m,dic_weight_decay[key]))
            if 'drop_rate' not in [sub_name[ii], sub_name[jj]]:
                co_drop_rate.append(kendall(test_m,dic_drop_rate[key]))
                m_i_drop_rate.append(mi(test_m,dic_drop_rate[key]))
            if 'width' not in [sub_name[ii], sub_name[jj]]:
                co_width.append(kendall(test_m,dic_width[key]))
                m_i_width.append(mi(test_m,dic_width[key]))
            if 'batch_size' not in [sub_name[ii], sub_name[jj]]:
                co_batch_size.append(kendall(test_m,dic_batch_size[key]))
                m_i_batch_size.append(mi(test_m,dic_batch_size[key]))
            if 'lr' not in [sub_name[ii], sub_name[jj]]:
                co_lr.append(kendall(test_m,dic_lr[key]))
                m_i_lr.append(mi(test_m,dic_lr[key]))

        #print(sub_name[ii]+'_'+sub_name[jj])
        #print('corr ',np.mean(co)) 
        #print('mi ',np.mean(m_i)) 
        all_co.append(np.mean(co))
        all_mi.append(np.mean(m_i))

        if len(co_act)>0:
            all_co_act.append(np.mean(co_act))
            all_mi_act.append(np.mean(m_i_act))
        if len(co_net)>0:
            all_co_net.append(np.mean(co_net))
            all_mi_net.append(np.mean(m_i_net))
        if len(co_weight_decay)>0:
            all_co_weight_decay.append(np.mean(co_weight_decay))
            all_mi_weight_decay.append(np.mean(m_i_weight_decay))
        if len(co_drop_rate)>0:
            all_co_drop_rate.append(np.mean(co_drop_rate))
            all_mi_drop_rate.append(np.mean(m_i_drop_rate))
        if len(co_width)>0:
            all_co_width.append(np.mean(co_width))
            all_mi_width.append(np.mean(m_i_width))
        if len(co_batch_size)>0:
            all_co_batch_size.append(np.mean(co_batch_size))
            all_mi_batch_size.append(np.mean(m_i_batch_size))
        if len(co_lr)>0:
            all_co_lr.append(np.mean(co_lr))
            all_mi_lr.append(np.mean(m_i_lr))

    pr_co = []
    pr_mi = []


    cco = np.mean(all_co)
    mmi = np.mean(all_mi) 
    pr_co.append(cco)
    pr_mi.append(mmi)

    return all_mi, pr_mi

In [5]:
#########w==0###################
def subset0(ij):
    if ij == 1:
        test_m = np.array(sum_f_norm)   
    elif ij == 2:
        test_m = np.array(sum_s_norm)
    elif ij == 3:
        test_m = np.array(sum_p_norm)
    elif ij == 4:
        test_m = np.array(path_norm)
    elif ij == 5:
        test_m = np.array(sigma)*2*np.log2(2*np.array(nop))
    elif ij == 6:
        test_m = np.array(sum_f_norm)/(np.array(sigma)*150000)
    elif ij == 7:
        test_m = np.array(sum_f_norm)/(np.array(sigma)*150000) + np.log2(1/np.array(laplace_det))
    elif ij == 8:
        test_m = np.array(sum_f_norm)/(np.array(sigma)*150000) + np.log2(1/np.array(sampling_det))

    return (mi(test_m,ge))

In [6]:
import numpy as np

def mi2(a,b):
    a = np.array(a)
    b = np.array(b)
    index = np.arange(len(a))
    np.random.shuffle(index)
    a = a[index]
    b = b[index]
    h1 = 0
    h2 = 0
    h3 = 0
    h4 = 0
    for i in range(len(a)-1):
        for j in range(i+1,len(a)):
            if (a[i]-a[j])>0 and (b[i]-b[j])>0:
                h1 += 1
            elif (a[i]-a[j])>0 and (b[i]-b[j])<0:
                h2 += 1
            elif (a[i]-a[j])<0 and (b[i]-b[j])>0:
                h3 += 1
            elif (a[i]-a[j])<0 and (b[i]-b[j])<0:
                h4 += 1
    h_all = h1+h2+h3+h4
    if h_all == 0:
        return 10
    else:
        p1 = h1/h_all
        p2 = h2/h_all
        p3 = h3/h_all
        p4 = h4/h_all
        mi = 0
        if p1>0.:
            mi += p1*np.log2(p1/((p1+p2)*(p1+p3))) 
        if p2>0.:
            mi += p2*np.log2(p2/((p1+p2)*(p2+p4))) 
        if p3>0.:
            mi += p3*np.log2(p3/((p3+p4)*(p1+p3))) 
        if p4>0.:
            mi += p4*np.log2(p4/((p3+p4)*(p2+p4)))
        return mi
    
def co_parameter(ii,jj,co):
    coo = []
    co_ = []
    tar = []
    for i in range(int(288/ii)):
        for j in range(int(ii/jj)):
            if jj==2:
                co_.append(co[i*ii+j])
                co_.append(co[int(i*ii+j+ii/jj)])
                tar.append(1)
                tar.append(2)
            if jj==3:
                co_.append(co[i*ii+j])
                co_.append(co[int(i*ii+j+ii/jj)])
                co_.append(co[int(i*ii+j+2*ii/jj)])
                tar.append(1)
                tar.append(2)
                tar.append(3)
            
            if len(co_)==12:
                cco = mi2(co_,tar)
                if cco<2:
                    coo.append(cco)
                co_ = []
                tar = []
    return np.mean(coo)

def subco(ij):
    co = []

    if ij == 1:
        test_m = np.array(sum_f_norm)   
    elif ij == 2:
        test_m = np.array(sum_s_norm)
    elif ij == 3:
        test_m = np.array(sum_p_norm)
    elif ij == 4:
        test_m = np.array(path_norm)
    elif ij == 5:
        test_m = np.array(sigma)*2*np.log2(2*np.array(nop))
    elif ij == 6:
        test_m = np.array(sum_f_norm)/(np.array(sigma)*150000)
    elif ij == 7:
        test_m = np.array(sum_f_norm)/(np.array(sigma)*150000) + np.log2(1/np.array(laplace_det))
    elif ij == 8:
        test_m = np.array(sum_f_norm)/(np.array(sigma)*150000) + np.log2(1/np.array(sampling_det))

    co.append(co_parameter(4,2,test_m))
    co.append(co_parameter(2,2,test_m))
    co.append(co_parameter(144,3,test_m))
    co.append(co_parameter(288,2,test_m))
    co.append(co_parameter(48,2,test_m))
    co.append(co_parameter(8,2,test_m))
    co.append(co_parameter(24,2,test_m))

    return co

In [7]:
import warnings
warnings.filterwarnings('ignore')

def prt(iii):
    if iii==1:
        print('& Frob Distance & ', end='')
    elif iii==2:
        print('& Spectral Norm & ', end='') 
    elif iii==3:
        print('& Parameter Norm & ', end='')
    elif iii==4:
        print('& Path Norm & ', end='')
    elif iii==5:
        print('& Sharpness $\\alpha\'$ & ', end='')
    elif iii==6:
        print('& \\textbf{Pac-Sharpness} & ', end='')
    elif iii==7:
        print('& \\textbf{Pac-S(Laplace)} & ', end='')
    elif iii==8:
        print('& \\textbf{Pac-S(Sampling)} & ', end='')
        
for iii in range(1,9):
    prt(iii)
    b = subco(iii)
    for x in b:
        print("%.4f" % x, end = ' & ')

    a2, b2 = subset2(iii)
    print("%.4f" % b2[0], end = ' & ')

    a1, b1 = subset1(iii)
    print("%.4f" % b1[0], end = ' & ')

    b0 = subset0(iii)
    print("%.4f" % b0, end='\\\\')
    print('')

print('############################################################################')
for iii in range(1,9):
    prt(iii)
    a2, b2 = subset2(iii)
    for jjj in range(len(a2)-1):
        print("%.4f" % a2[jjj], end = ' & ')
    print("%.4f" % a2[-1], end='\\\\')
    print('')

print('')
print('############################################################################')
for iii in range(1,9):
    prt(iii)
    a1, b1 = subset1(iii)
    for jjj in range(len(a1)-1):
        print("%.4f" % a1[jjj], end = ' & ')
    print("%.4f" % a1[-1], end='\\\\')
    print('')

& Frob Distance & 0.1832 & 0.2565 & 0.3705 & 0.0732 & 0.1677 & 0.4874 & 0.0644 & 0.0289 & 0.0159 & 0.0094\\
& Spectral Norm & 0.2527 & 0.3155 & 0.2482 & 0.1090 & 0.7924 & 0.2163 & 0.1259 & 0.0937 & 0.0932 & 0.0986\\
& Parameter Norm & 0.2461 & 0.3109 & 0.1363 & 0.1039 & 0.8968 & 0.1982 & 0.0583 & 0.0927 & 0.0865 & 0.0826\\
& Path Norm & 0.2325 & 0.1854 & 0.2543 & 0.5054 & 0.2787 & 0.2073 & 0.1530 & 0.0425 & 0.0307 & 0.0177\\
& Sharpness $\alpha'$ & 0.1238 & 0.0728 & 0.2038 & 0.5561 & 0.3633 & 0.3128 & 0.1583 & 0.0877 & 0.0639 & 0.0418\\
& \textbf{Pac-Sharpness} & 0.1518 & 0.3286 & 0.0468 & 0.5185 & 0.4043 & 0.1112 & 0.1277 & 0.0552 & 0.0324 & 0.0097\\
& \textbf{Pac-S(Laplace)} & 0.0630 & 0.1164 & 0.0805 & 0.2851 & 0.3425 & 0.0306 & 0.7325 & 0.1248 & 0.1123 & 0.1052\\
& \textbf{Pac-S(Sampling)} & 0.3059 & 0.1739 & 0.2552 & 0.2163 & 0.3663 & 0.1382 & 0.4299 & 0.1012 & 0.0712 & 0.0466\\
############################################################################
& Frob Distance & 0.0205 &